# BERGAMINI FEDERICO 845646

Convertitore da FASTQ a FASTA.

**get_quality_percentage(quality_pherd_string, quality_threshold)**
    
    Prende in input la sequenza di phred value relativi ad un record fastq e una soglia minimima.
    Restituisce la percentuale di caratteri almeno pari alla soglia minima.

In [1]:
def get_quality_percentage(quality_pherd_string, quality_threshold):
    return len([phred_value for phred_value in quality_pherd_string if phred_value >= quality_threshold]) / len(quality_pherd_string)

**def get_trimming_interval(quality_string, quality_threshold):**

    Prende in input la sequenza di phred value relativi ad un record fastq e una soglia minimima.
    Restituisce il massimo intervallo di valori consecutivi almeno pari alla soglia.

In [2]:
def get_trimming_interval(quality_string, quality_threshold):
    bool_list = [(phred_value >= quality_threshold) for phred_value in quality_string]
    
    start_list = []
    for i in range(len(bool_list))[1:]:
        if bool_list[i] == True and bool_list[i-1] == False:
            start_list.append(i)
        
    if bool_list[0] == True:
        start_list[:0] = [0]
        
    start_list[:0] = [1]
    
    end_list = []
    for j in range(len(bool_list))[:-1]:
        if bool_list[j] == True and bool_list[j+1] == False:
            end_list.append(j)
        
    if bool_list[-1] == True:
        end_list[len(bool_list):] = [len(bool_list)-1]
        
    end_list[:0] = [0]
    
    interval_lengths = [end_list[i]-start_list[i]+1 for i in range(len(start_list))]
    
    trimming_index = interval_lengths.index(max(interval_lengths))
    interval_start = start_list[trimming_index]
    interval_end = end_list[trimming_index]
    
    return [interval_start, interval_end + 1]

Importo BioPython.

In [3]:
import Bio

Importo SeqIO.

In [4]:
from Bio import SeqIO

Inserimento dei parametri di input.

In [5]:
l1 = 20
l2 = 40#200
q1 = 35#20
q2 = 40#60
perc_p = 0.5#0.1

Leggo il file FASTQ usando BioPython.

In [6]:
fastq_records = SeqIO.parse("input.fq", "fastq")

Estraggo i record.

In [7]:
fastq_record_list = list(fastq_records)

Stampo i record.

In [8]:
fastq_record_list

[SeqRecord(seq=Seq('TATGGAGGCCCAACTTCTTGTATTCACAGGTTCTGC'), id='HWUSI-EAS522:8:5:662:692#0/1', name='HWUSI-EAS522:8:5:662:692#0/1', description='HWUSI-EAS522:8:5:662:692#0/1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTGCCAACTTCTTATGGAGGCCTGTATTCACAGGT'), id='HWUSI-EAS522:8:5:662:693#0/1', name='HWUSI-EAS522:8:5:662:693#0/1', description='HWUSI-EAS522:8:5:662:693#0/1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTGCCAGAGGCCTGTATTCACAGGTACTTCTTATG'), id='HWUSI-EAS522:8:5:662:694#0/1', name='HWUSI-EAS522:8:5:662:694#0/1', description='HWUSI-EAS522:8:5:662:694#0/1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCGCCTGTATTCACAGGTTGCCAACTTCTTATGGAG'), id='HWUSI-EAS522:8:5:662:695#0/1', name='HWUSI-EAS522:8:5:662:695#0/1', description='HWUSI-EAS522:8:5:662:695#0/1', dbxrefs=[])]

Rimuovo i read di lunghezza non compresa tra l1 e l2

In [9]:
fastq_record_list = [read for read in fastq_record_list if len(read.seq) < l2 and len(read.seq) > l1]

In [10]:
fastq_record_list

[SeqRecord(seq=Seq('TATGGAGGCCCAACTTCTTGTATTCACAGGTTCTGC'), id='HWUSI-EAS522:8:5:662:692#0/1', name='HWUSI-EAS522:8:5:662:692#0/1', description='HWUSI-EAS522:8:5:662:692#0/1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTGCCAACTTCTTATGGAGGCCTGTATTCACAGGT'), id='HWUSI-EAS522:8:5:662:693#0/1', name='HWUSI-EAS522:8:5:662:693#0/1', description='HWUSI-EAS522:8:5:662:693#0/1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTGCCAGAGGCCTGTATTCACAGGTACTTCTTATG'), id='HWUSI-EAS522:8:5:662:694#0/1', name='HWUSI-EAS522:8:5:662:694#0/1', description='HWUSI-EAS522:8:5:662:694#0/1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCGCCTGTATTCACAGGTTGCCAACTTCTTATGGAG'), id='HWUSI-EAS522:8:5:662:695#0/1', name='HWUSI-EAS522:8:5:662:695#0/1', description='HWUSI-EAS522:8:5:662:695#0/1', dbxrefs=[])]

Rimuovo i read con basi di qualità inferiore a q1

In [11]:
fastq_record_list = [read for read in fastq_record_list if get_quality_percentage(read.letter_annotations['phred_quality'], q1) == 1]

In [12]:
fastq_record_list

[SeqRecord(seq=Seq('TATGGAGGCCCAACTTCTTGTATTCACAGGTTCTGC'), id='HWUSI-EAS522:8:5:662:692#0/1', name='HWUSI-EAS522:8:5:662:692#0/1', description='HWUSI-EAS522:8:5:662:692#0/1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTGCCAGAGGCCTGTATTCACAGGTACTTCTTATG'), id='HWUSI-EAS522:8:5:662:694#0/1', name='HWUSI-EAS522:8:5:662:694#0/1', description='HWUSI-EAS522:8:5:662:694#0/1', dbxrefs=[])]

Rimuovo i record che non contengono una sottoregione almeno lunga p% della lunghezza del record con qualità almeno q2

In [13]:
tmp = []
for read in fastq_record_list:
    trimming_interval = get_trimming_interval(read.letter_annotations['phred_quality'], q2)
    if (trimming_interval[1] - trimming_interval[0]) >= (perc_p * len(read.seq)):
        tmp.append(read);

fastq_record_list = tmp;

Completo l'header fasta prima della stampa aggiungendo le informazioni richieste.

In [14]:
for read in fastq_record_list:
    phred_values = read.letter_annotations['phred_quality']
    length = len(read.seq)
    min_base_quality = min(phred_values)
    min_q2_interval = get_trimming_interval(phred_values, q2)
    medium_quality_q2 = sum(phred_values[min_q2_interval[0]: min_q2_interval[1]]) / length
    read.description = "length : " + str(length) + " min_base_quality : " + str(min_base_quality) + " start_end_min_q2 : " + str(min_q2_interval) + " medium_quality_q2 : " + str(medium_quality_q2) 

Cambio formattazione in FASTA

In [15]:
fasta_record_list = [read.format("fasta") for read in fastq_record_list] 
fasta_record_list

['>HWUSI-EAS522:8:5:662:692#0/1 length : 36 min_base_quality : 48 start_end_min_q2 : [0, 36] medium_quality_q2 : 60.416666666666664\nTATGGAGGCCCAACTTCTTGTATTCACAGGTTCTGC\n',
 '>HWUSI-EAS522:8:5:662:694#0/1 length : 36 min_base_quality : 48 start_end_min_q2 : [0, 36] medium_quality_q2 : 65.75\nTCTGCCAGAGGCCTGTATTCACAGGTACTTCTTATG\n']

Stampo su file

In [16]:
with open ("output.fa", "w") as output_file:
    for read in fasta_record_list:
        output_file.write(read)